In [1]:
database='D:/Cryptography/Deeplearning/NLP/extracted/AA/wiki_00'

In [2]:
import os

In [3]:
os.path.exists(database)

True

In [4]:
import pandas as pd

In [5]:
dataframe= open(database, 'r',encoding='UTF-8')

In [6]:
DATA=dataframe.read()

In [7]:
DATA

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [8]:
len(DATA)

38877318

In [9]:
from hanziconv import HanziConv

In [10]:
data=HanziConv.toSimplified(DATA)

In [11]:
data

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [12]:
import re

In [13]:
def token(string):
    return ' '.join(re.findall('[\w|\d]+',string))

In [14]:
all_articles=token(data)

In [15]:
import jieba

In [16]:
def cut(string): return list(jieba.cut(string))

In [17]:
ALL_TOKENS = cut(all_articles)

Building prefix dict from the default dictionary ...
Loading model from cache X:\Users\SHIYIN~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.039 seconds.
Prefix dict has been built succesfully.


In [18]:
valida_tokens = [t for t in ALL_TOKENS if t.strip() and t!='n']

In [19]:
len(ALL_TOKENS)

21192767

In [20]:
len(valida_tokens)

17408696

In [21]:
from collections import Counter

In [22]:
words_count = Counter(valida_tokens)

In [23]:
words_count.most_common(10)

[('的', 996054),
 ('在', 258309),
 ('年', 209677),
 ('是', 184779),
 ('和', 159641),
 ('了', 126933),
 ('为', 117140),
 ('与', 92352),
 ('有', 88780),
 ('月', 86990)]

In [24]:
frequences_all = [f for w, f in words_count.most_common()]

In [25]:
frequences_sum = sum(frequences_all)

In [26]:
frequences_sum

17408696

In [27]:
1 / frequences_sum

5.744255629485402e-08

In [28]:
def get_prob(word):
    esp = 1/ frequences_sum
    if word in words_count:
        return words_count[word] / frequences_sum
    else:
        return esp

In [29]:
get_prob('青岛')

5.330669224162453e-05

In [30]:
valid_tokens = [str(t) for t in valida_tokens]

In [31]:
all_2_grams_words = [''.join(valid_tokens[i:i+2]) for i in range(len(valid_tokens[:-2]))]

In [32]:
_2_gram_sum = len(all_2_grams_words)
_2_gram_counter = Counter(all_2_grams_words)

def get_combination_prob(w1, w2):
    if w1+ w2 in _2_gram_counter: return _2_gram_counter[w1 + w2] / _2_gram_sum
    else:
        return 1 / _2_gram_sum

In [33]:
get_combination_prob('去','北京')

6.318681918356425e-07

In [34]:
def get_prob_2_gram(w1, w2):
    return get_combination_prob(w1, w2) / get_prob(w1)

In [35]:
def language_model_of_2_gram(sentences):
    sentence_probability = 1
    
    words = cut(sentences)
    
    
    for i, word in enumerate(words):
        if i== 0:
            prob = get_prob(word)
        else:
            previous = words[i-1]
            prob = get_prob_2_gram(previous, word)
        
        sentence_probability *= prob
    
    return sentence_probability
        

In [36]:
language_model_of_2_gram('小明今天抽奖抽到一台苹果手机')

1.4646772292598628e-18

In [37]:
language_model_of_2_gram('小明今天抽奖抽到一台波音飞机')

2.045163278209176e-17

In [44]:
need_compared= [
    "今天晚上请你吃大餐，我们一起吃日料 明天晚上请你吃大餐，我们一起吃苹果",
    "真事一直好看的小猫 真是一只好看的小猫",
    "今晚火锅去吃我 今晚我去吃火锅",
    "今天晚上我们考试 今天晚上我们考室",
    "这这是一个常见的并举 这是一个常见的病句",
    "人民解放军的勇敢是世界一流军队 人们解放军是世界一流的勇敢军队"
]

for s in need_compared:
    s1, s2 = s.split()
    p1, p2 = language_model_of_2_gram(s1),language_model_of_2_gram(s2)
    
    better = s1 if p1 > p2 else s2
    
    print('{} is more possible'.format(better))
    print('-'*4 + ' {} wirh probility {}'.format(s1,p1))
    print('-'*4 + ' {} wirh probility {}'.format(s2,p2))    

今天晚上请你吃大餐，我们一起吃日料 is more possible
---- 今天晚上请你吃大餐，我们一起吃日料 wirh probility 2.5128804936265445e-26
---- 明天晚上请你吃大餐，我们一起吃苹果 wirh probility 5.025760987253087e-27
真是一只好看的小猫 is more possible
---- 真事一直好看的小猫 wirh probility 9.645634940491874e-21
---- 真是一只好看的小猫 wirh probility 1.889404925819887e-16
今晚我去吃火锅 is more possible
---- 今晚火锅去吃我 wirh probility 1.1021904223122089e-15
---- 今晚我去吃火锅 wirh probility 2.68342091826396e-14
今天晚上我们考室 is more possible
---- 今天晚上我们考试 wirh probility 7.222690312968652e-14
---- 今天晚上我们考室 wirh probility 7.222690312968652e-14
这是一个常见的病句 is more possible
---- 这这是一个常见的并举 wirh probility 3.387259796229185e-19
---- 这是一个常见的病句 wirh probility 1.3881485340690377e-14
人民解放军的勇敢是世界一流军队 is more possible
---- 人民解放军的勇敢是世界一流军队 wirh probility 2.9171795989719258e-21
---- 人们解放军是世界一流的勇敢军队 wirh probility 5.631265062685468e-23
